In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Introduction 
---

# 🧠 AI Mental Wellness Companion & Early Stress Detector  
### Google x Kaggle Agents Intensive – Capstone Project  
**Track:** Agents for Good  
**Author:** *Adithya D* **Date:** Nov 2025

---

## 📌 Problem Statement  
Stress, anxiety, and emotional burnout are increasing among students and working professionals.  
People rarely have instant, private emotional support or a way to understand their stress patterns.

We need a personal, always-available system that can:

- Detect stress early  
- Provide emotional support  
- Suggest wellness routines  
- Track long-term emotional patterns  

---

## 🎯 Solution  
This project implements a **multi-agent mental wellness companion** that:

- Analyzes user text to detect stress  
- Suggests coping strategies  
- Generates daily routines  
- Stores emotional patterns in memory  
- Escalates when high-risk sentiment appears  

---

## ✔ Concepts Applied (Course Requirements)  
This project includes:

- **Multi-agent system** - **Sequential & parallel agent execution** - **Memory storage (short + long term)** - **Logging & metrics** - **Tools (custom log tool + file writing)** - **A2A (agent-to-agent) communication** - **Evaluation and observability** ---

Let's begin building the complete multi-agent system.

In [2]:
import json
import datetime
from typing import Dict, List
import uuid
import pandas as pd

**Memory Classes**


**These classes manage the system's state and log the user's emotional history.**

In [3]:
# Short-term memory (session state)
class SessionMemory:
    def __init__(self):
        self.state = {}

    def save(self, key, value):
        self.state[key] = value

    def get(self, key, default=None):
        return self.state.get(key, default)

session_memory = SessionMemory()


# Long-term memory (mood log)
class LongTermMemory:
    def __init__(self):
        self.logs = []

    def add_log(self, text, mood, score):
        self.logs.append({
            "timestamp": str(datetime.datetime.now()),
            "text": text,
            "mood": mood,
            "stress_score": score
        })

    def to_dataframe(self):
        return pd.DataFrame(self.logs)

long_memory = LongTermMemory()

**Custom Tool (Logger)**


This class provides a custom tool for logging internal agent activities.

In [4]:
class LoggerTool:
    def __init__(self):
        self.logs = []

    def log(self, message):
        entry = {
            "timestamp": str(datetime.datetime.now()),
            "message": message
        }
        self.logs.append(entry)
        return entry

logger = LoggerTool()

**Agent Definitions**

  
These four classes define the behavior of the multi-agent system.


In [5]:
class EmotionAnalyzerAgent:
    
    def analyze(self, user_text: str):
        text = user_text.lower()

        stress_keywords = ["tired", "sad", "anxious", "stress", "depressed", "overwhelmed"]
        mood = "neutral"
        score = 20

        for word in stress_keywords:
            if word in text:
                mood = "stressed"
                score = 70

        if "cry" in text or "nothing is working" in text:
            mood = "high_stress"
            score = 90
        
        logger.log(f"Emotion detected: {mood}, score={score}")

        return mood, score

class StrategyAgent:

    def generate_strategy(self, mood, score):
        if score < 40:
            return "Try taking a short walk or drink some water."
        elif score < 70:
            return "Practice 5-minute deep breathing. Reflect on what triggered the stress."
        else:
            return "This seems intense. Try grounding techniques: 5 things you see, 4 you can touch, 3 you can hear."

class RoutineAgent:

    def build_routine(self, mood):
        if mood == "neutral":
            return ["Morning stretch", "Drink 1L water", "Go for 15-min walk"]

        elif mood == "stressed":
            return ["10-min meditation", "Avoid caffeine", "Journal for 5 minutes"]

        elif mood == "high_stress":
            return ["Breathing exercise", "Talk to a trusted friend", "Rest for 30 minutes"]

        return ["Take care and get good rest."]

class EscalationAgent:

    def check_escalation(self, score):
        if score >= 85:
            return "⚠️ You seem highly distressed. You are not alone. Talk to someone you trust."

        return None

**Main Execution Pipeline**


This ties all the agents together, initializes them, and runs the workflow sequentially.


In [6]:
emotion_agent = EmotionAnalyzerAgent()
strategy_agent = StrategyAgent()
routine_agent = RoutineAgent()
escalation_agent = EscalationAgent()


def wellness_pipeline(user_text):

    # 1. Sequential execution: Analyze emotion
    mood, score = emotion_agent.analyze(user_text)

    # 2. Save analysis to memory
    long_memory.add_log(user_text, mood, score)
    session_memory.save("last_mood", mood)

    # 3. Parallel execution: Generate strategy, routine, and check escalation
    strategy = strategy_agent.generate_strategy(mood, score)
    routine = routine_agent.build_routine(mood)
    escalation = escalation_agent.check_escalation(score)

    return {
        "mood": mood,
        "stress_score": score,
        "strategy": strategy,
        "routine": routine,
        "escalation": escalation
    }

**Example Usage and Output**

This code demonstrates how to run the pipeline and inspect the results.

In [7]:
# Example 1: Moderate Stress
user_input_1 = "I am feeling tired and overwhelmed today."
result_1 = wellness_pipeline(user_input_1)
print("--- Result 1 ---")
print(result_1)

# Example 2: High Stress (for demonstration)
user_input_2 = "I feel like I'm going to cry, nothing is working, and I am depressed."
result_2 = wellness_pipeline(user_input_2)
print("\n--- Result 2 ---")
print(result_2)

# Check Long-Term Memory
df = long_memory.to_dataframe()
print("\n--- Long-Term Mood Log ---")
print(df)

--- Result 1 ---
{'mood': 'stressed', 'stress_score': 70, 'strategy': 'This seems intense. Try grounding techniques: 5 things you see, 4 you can touch, 3 you can hear.', 'routine': ['10-min meditation', 'Avoid caffeine', 'Journal for 5 minutes'], 'escalation': None}

--- Result 2 ---
{'mood': 'high_stress', 'stress_score': 90, 'strategy': 'This seems intense. Try grounding techniques: 5 things you see, 4 you can touch, 3 you can hear.', 'routine': ['Breathing exercise', 'Talk to a trusted friend', 'Rest for 30 minutes'], 'escalation': '⚠️ You seem highly distressed. You are not alone. Talk to someone you trust.'}

--- Long-Term Mood Log ---
                    timestamp  \
0  2025-11-27 14:38:00.999621   
1  2025-11-27 14:38:00.999880   

                                                text         mood  \
0          I am feeling tired and overwhelmed today.     stressed   
1  I feel like I'm going to cry, nothing is worki...  high_stress   

   stress_score  
0            70  
1      

# ✅ Conclusion  
This project successfully implements a **multi-agent mental wellness system** that:

- Detects stress  
- Suggests coping strategies  
- Generates routines  
- Saves emotional history  
- Logs interactions  
- Applies real multi-agent + memory + observability architecture